---
### Limpieza y depuración de datos de peleas

En esta sección se documentan los pasos realizados para limpiar, transformar y depurar el dataset de peleas. Se explican las decisiones tomadas y se justifica cada transformación aplicada para asegurar la calidad y consistencia de los datos.

In [37]:
# 1. Carga de datos y librerías
import pandas as pd
import numpy as np

# Cargar el archivo raw_fights.csv
df = pd.read_csv('../data/raw/raw_fights.csv')

In [38]:
# 2. Visualización de datos brutos
print(f"Filas: {len(df)} | Columnas: {list(df.columns)}")
df.head()

Filas: 8351 | Columnas: ['event_id', 'fight_id', 'fight_order', 'red_id', 'red_name', 'blue_id', 'blue_name', 'winner_id', 'weight_class', 'referee', 'round', 'time', 'time_format', 'method', 'details', 'bonus', 'kd1', 'kd2', 'str1', 'str2', 'td1', 'td2', 'sub1', 'sub2', 'control_time1', 'control_time2', 'sig_head1', 'sig_head2', 'sig_body1', 'sig_body2', 'sig_leg1', 'sig_leg2', 'total_str1', 'total_str2', 'pass1', 'pass2', 'rev1', 'rev2']


,event_id,fight_id,fight_order,red_id,red_name,blue_id,blue_name,winner_id,weight_class,referee,...,sig_body1,sig_body2,sig_leg1,sig_leg2,total_str1,total_str2,pass1,pass2,rev1,rev2
0,5efaaf313b652dd7,de1a3734be60e6a1,1,f166e93d04a8c274,Diego Lopes,52ef95b5860fb28c,Jean Silva,f166e93d04a8c274,Featherweight Bout,Mike Beltran,...,1 of 1,10 of 17,10 of 16,7 of 7,86 of 154,43 of 91,0,0,0,1
1,5efaaf313b652dd7,944b929130b23c2a,2,05339613bf8e9808,Rob Font,9a97acbfd5a08bfa,David Martinez,9a97acbfd5a08bfa,Bantamweight Bout,Dan Miragliotta,...,7 of 17,10 of 16,2 of 3,19 of 24,49 of 158,115 of 210,0,0,0,0
2,5efaaf313b652dd7,7cbfeba85f86d1bf,3,e132d47bd9efbbe0,Rafa Garcia,7026eca45f65377b,Jared Gordon,e132d47bd9efbbe0,Lightweight Bout,Kerry Hatley,...,7 of 7,8 of 13,10 of 11,2 of 2,107 of 179,80 of 201,0,0,1,0
3,5efaaf313b652dd7,114d2f6fcd3f6f00,4,8c0580d4fff106c1,Kelvin Gastelum,71505842fb6455c3,Dustin Stoltzfus,8c0580d4fff106c1,Middleweight Bout,Jacob Montalvo,...,16 of 24,25 of 34,11 of 12,14 of 19,58 of 118,69 of 151,0,0,0,1
4,5efaaf313b652dd7,202c47db69768356,5,262a7d06203657e6,Alexander Hernandez,f53d4f21d1b5f2dc,Diego Ferreira,262a7d06203657e6,Lightweight Bout,Jeff Rexroad,...,4 of 6,11 of 35,4 of 6,4 of 7,30 of 75,23 of 98,0,0,0,0


In [39]:
# 3. Limpieza de columnas de texto (nombres, métodos, detalles, árbitro, etc.)
def clean_text(val):
    if pd.isna(val):
        return val
    val = str(val).strip()
    val = ' '.join(val.split())
    return val.title()

text_cols = ['red_name', 'blue_name', 'winner_id', 'referee', 'method', 'details']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

df.head()

,event_id,fight_id,fight_order,red_id,red_name,blue_id,blue_name,winner_id,weight_class,referee,...,sig_body1,sig_body2,sig_leg1,sig_leg2,total_str1,total_str2,pass1,pass2,rev1,rev2
0,5efaaf313b652dd7,de1a3734be60e6a1,1,f166e93d04a8c274,Diego Lopes,52ef95b5860fb28c,Jean Silva,F166E93D04A8C274,Featherweight Bout,Mike Beltran,...,1 of 1,10 of 17,10 of 16,7 of 7,86 of 154,43 of 91,0,0,0,1
1,5efaaf313b652dd7,944b929130b23c2a,2,05339613bf8e9808,Rob Font,9a97acbfd5a08bfa,David Martinez,9A97Acbfd5A08Bfa,Bantamweight Bout,Dan Miragliotta,...,7 of 17,10 of 16,2 of 3,19 of 24,49 of 158,115 of 210,0,0,0,0
2,5efaaf313b652dd7,7cbfeba85f86d1bf,3,e132d47bd9efbbe0,Rafa Garcia,7026eca45f65377b,Jared Gordon,E132D47Bd9Efbbe0,Lightweight Bout,Kerry Hatley,...,7 of 7,8 of 13,10 of 11,2 of 2,107 of 179,80 of 201,0,0,1,0
3,5efaaf313b652dd7,114d2f6fcd3f6f00,4,8c0580d4fff106c1,Kelvin Gastelum,71505842fb6455c3,Dustin Stoltzfus,8C0580D4Fff106C1,Middleweight Bout,Jacob Montalvo,...,16 of 24,25 of 34,11 of 12,14 of 19,58 of 118,69 of 151,0,0,0,1
4,5efaaf313b652dd7,202c47db69768356,5,262a7d06203657e6,Alexander Hernandez,f53d4f21d1b5f2dc,Diego Ferreira,262A7D06203657E6,Lightweight Bout,Jeff Rexroad,...,4 of 6,11 of 35,4 of 6,4 of 7,30 of 75,23 of 98,0,0,0,0


In [40]:
# 4. Conversión y limpieza de columnas numéricas
num_cols = [
    'kd1', 'kd2', 'str1', 'str2', 'td1', 'td2', 'sub1', 'sub2',
    'control_time1', 'control_time2', 'sig_head1', 'sig_head2',
    'sig_body1', 'sig_body2', 'sig_leg1', 'sig_leg2',
    'total_str1', 'total_str2', 'pass1', 'pass2', 'rev1', 'rev2',
    'round'
]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df[num_cols].describe()

,kd1,kd2,str1,str2,td1,td2,sub1,sub2,control_time1,control_time2,...,sig_body2,sig_leg1,sig_leg2,total_str1,total_str2,pass1,pass2,rev1,rev2,round
count,8351.000000,8351.000000,21.0,21.0,21.0,21.0,8351.000000,8351.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8351.0,8351.0,8351.000000,8351.000000,8330.000000
mean,0.243564,0.182852,0.0,0.0,0.0,0.0,0.435996,0.317926,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.134714,0.134116,2.356663
std,0.515228,0.462457,0.0,0.0,0.0,0.0,0.875450,0.746321,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.425736,0.420691,1.016927
min,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,0.000000,3.000000
75%,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,0.000000,3.000000
max,5.000000,6.000000,0.0,0.0,0.0,0.0,10.000000,7.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6.000000,5.000000,5.000000


In [41]:
# 5. Limpieza y estandarización de columnas de tiempo
def parse_time(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, (int, float)):
        return val
    val = str(val).strip()
    if ':' in val:
        try:
            parts = val.split(':')
            if len(parts) == 2:
                return int(parts[0]) * 60 + float(parts[1])
            elif len(parts) == 3:
                return int(parts[0]) * 3600 + int(parts[1]) * 60 + float(parts[2])
        except Exception:
            return np.nan
    try:
        return float(val)
    except Exception:
        return np.nan

for col in ['time', 'control_time1', 'control_time2']:
    if col in df.columns:
        df[col] = df[col].apply(parse_time)

df[['time', 'control_time1', 'control_time2']].head()

,time,control_time1,control_time2
0,288.0,NaN,NaN
1,300.0,NaN,NaN
2,147.0,NaN,NaN
3,300.0,NaN,NaN
4,226.0,NaN,NaN


In [42]:
# 6. Limpieza de weight_class (integrada desde fights_weightcleaning)
# Paso 1: Renombrar primero las categorías femeninas a un nombre temporal único
def temp_women_weight_class(w):
    if pd.isnull(w):
        return w
    if "women" in w.lower() and "bantamweight" in w.lower():
        return "Women's B"
    if "women" in w.lower() and "featherweight" in w.lower():
        return "Women's F"
    if "women" in w.lower() and "flyweight" in w.lower():
        return "Women's Fly"
    if "women" in w.lower() and "strawweight" in w.lower():
        return "Women's Straw"
    return w
if 'weight_class' in df.columns:
    df['weight_class'] = df['weight_class'].apply(temp_women_weight_class)

# Paso 2: Normalizar categorías masculinas y especiales
male_categories = {
    "Flyweight": "Flyweight",
    "Bantamweight": "Bantamweight",
    "Featherweight": "Featherweight",
    "Lightweight": "Lightweight",
    "Welterweight": "Welterweight",
    "Middleweight": "Middleweight",
    "Light Heavyweight": "Light Heavyweight",
    "Heavyweight": "Heavyweight",
    "Open Weight": "Open Weight",
    "Catch Weight": "Catchweight",
    "Championship Bout": "Open Weight",
    "Tournament Title Bout": "Open Weight",
}
def clean_weight_class(w):
    if pd.isnull(w):
        return w
    for cat, label in male_categories.items():
        if cat.lower() in w.lower():
            return label
    return w
if 'weight_class' in df.columns:
    df['weight_class'] = df['weight_class'].apply(clean_weight_class)

# Paso 3: Volver a renombrar los temporales a su nombre final
final_map = {
    "Women's B": "Women's Bantamweight",
    "Women's F": "Women's Featherweight",
    "Women's Fly": "Women's Flyweight",
    "Women's Straw": "Women's Strawweight"
}
if 'weight_class' in df.columns:
    df['weight_class'] = df['weight_class'].replace(final_map)

df['weight_class'].value_counts(dropna=False)

weight_class
Lightweight              1410
Welterweight             1358
Middleweight             1100
Featherweight             819
Heavyweight               744
Bantamweight              741
Light Heavyweight         733
Flyweight                 389
Women's Strawweight       348
Women's Flyweight         260
Women's Bantamweight      230
Open Weight               116
Catchweight                73
Women's Featherweight      30
Name: count, dtype: int64

In [43]:
# 7. Limpieza de columnas de resultados y métodos de victoria
# Estandarizar 'method', 'details', 'winner_id'
def clean_method(val):
    if pd.isna(val):
        return val
    val = str(val).strip().lower()
    # Normalizar algunos métodos comunes
    mapping = {
        'ko/tko': 'KO/TKO',
        'submission': 'Submission',
        'decision - unanimous': 'Decision Unanimous',
        'decision - split': 'Decision Split',
        'decision - majority': 'Decision Majority',
        'dq': 'DQ',
        'draw': 'Draw',
        'no contest': 'No Contest',
    }
    for k, v in mapping.items():
        if k in val:
            return v
    return val.title()

for col in ['method', 'details']:
    if col in df.columns:
        df[col] = df[col].apply(clean_method)

if 'winner_id' in df.columns:
    df['winner_id'] = df['winner_id'].replace({'': np.nan, 'None': np.nan})

df[['method', 'details', 'winner_id']].head()

,method,details,winner_id
0,KO/TKO,Punches To Head On Ground,F166E93D04A8C274
1,Decision Unanimous,Chris Lee28 - 29.Sal D'Amato28 - 29.Junichiro ...,9A97Acbfd5A08Bfa
2,KO/TKO,Elbows To Head From Side Control,E132D47Bd9Efbbe0
3,Decision Unanimous,Chris Lee27 - 30.Jeff Rexroad28 - 29.Junichiro...,8C0580D4Fff106C1
4,KO/TKO,Punches To Head On Ground,262A7D06203657E6


In [44]:
# 8. Revisión de duplicados y registros nulos
# Eliminar duplicados por fight_id si existe
if 'fight_id' in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=['fight_id'])
    print(f"Eliminados {before - len(df)} duplicados por fight_id")

# Revisar nulos críticos (event_id, fight_id, red_id, blue_id)
critical_cols = ['event_id', 'fight_id', 'red_id', 'blue_id']
print(df[critical_cols].isnull().sum())
# Eliminar filas con nulos críticos
df = df.dropna(subset=critical_cols)
print(f"Filas tras eliminar nulos críticos: {len(df)}")

Eliminados 0 duplicados por fight_id
event_id    0
fight_id    0
red_id      0
blue_id     0
dtype: int64
Filas tras eliminar nulos críticos: 8351


In [45]:
# 9. Carga de datos de luchadores y eventos (Preparados para ML)

# Cargar los datasets ya limpios y transformados de los notebooks 02 y 03
try:
    # Se asume que guardaste estos archivos en '../data/ml/'
    fighters_ml = pd.read_csv('../data/ml/fighters.csv', index_col=0)
    events_ml = pd.read_csv('../data/ml/events.csv', index_col=0)
    
    print("Datos de luchadores y eventos (ML) cargados correctamente.")
    
    # Preparación de IDs para Merge
    # Renombrar 'fighter_id' a 'id' para la fusión
    fighters_ml = fighters_ml.rename(columns={'fighter_id': 'id'})
    # Quitar columnas no deseadas o redundantes de luchadores
    cols_to_use_fighters = [col for col in fighters_ml.columns if col not in ['first', 'last', 'nickname', 'dob']]
    
except FileNotFoundError as e:
    print(f"ERROR: No se pudo cargar un archivo necesario para ML: {e}. Asegúrate de ejecutar 02_fighters_cleaning.ipynb y 03_events_cleaning.ipynb y guardar los archivos en ../data/ml/")
    # Crear DataFrames vacíos para evitar que el notebook falle, aunque el resultado de ML será incompleto
    fighters_ml = pd.DataFrame(columns=['id'])
    events_ml = pd.DataFrame(columns=['event_id'])
    cols_to_use_fighters = []
    
# Definir el DataFrame limpio de peleas como 'ml_df' para continuar
ml_df = df.copy() # df contiene todas tus columnas limpias hasta el momento


Datos de luchadores y eventos (ML) cargados correctamente.


In [46]:
# 9. Fusión de características de Luchadores y Eventos

# --- 9.1 Fusión de Luchador ROJO (red_id) ---
ml_df = pd.merge(ml_df, fighters_ml[cols_to_use_fighters],
                 left_on='red_id', right_on='fighter_id', how='left', suffixes=('', '_red'))
# Renombrar columnas de luchador fusionadas para el córner ROJO
ml_df = ml_df.rename(columns={col: col + '_red' for col in cols_to_use_fighters if col != 'id'})
ml_df = ml_df.drop(columns=['id_red'], errors='ignore') # Eliminar ID duplicado tras merge

# --- 9.2 Fusión de Luchador AZUL (blue_id) ---
ml_df = pd.merge(ml_df, fighters_ml[cols_to_use_fighters],
                 left_on='blue_id', right_on='fighter_id', how='left', suffixes=('', '_blue'))
# Renombrar columnas de luchador fusionadas para el córner AZUL
ml_df = ml_df.rename(columns={col: col + '_blue' for col in cols_to_use_fighters if col != 'id'})
ml_df = ml_df.drop(columns=['id_blue'], errors='ignore') # Eliminar ID duplicado tras merge

# --- 9.3 Definición de la Variable Target ---
# 1. Quitar filas donde no hay ganador (NC, Draw)
ml_df = ml_df.dropna(subset=['winner_id'])
# 2. Crear Target: 1 si el peleador Rojo ganó, 0 si el Azul ganó
ml_df['target_red'] = np.where(ml_df['red_id'] == ml_df['winner_id'], 1, 0)

print(f"Dimensiones tras merge y definición de target: {ml_df.shape}")


Dimensiones tras merge y definición de target: (8203, 75)


In [47]:
# 10. Creación del Dataset Simétrico (Preparación Final para ML)

# 10.1. Identificar columnas de características por córner
# Las estadísticas de pelea post-fight (kd1, str_acc1, etc.)
stats_cols = [col.split('1')[0] for col in ml_df.columns if col.endswith('1')]
# Las características de luchador pre-fight (height_red, slpm_red, etc.)
fighter_stat_cols = [col for col in ml_df.columns if col.endswith('_red') or col.endswith('_blue')]

red_cols = [col for col in ml_df.columns if col.endswith('_red') or col.endswith('1')]
blue_cols = [col for col in ml_df.columns if col.endswith('_blue') or col.endswith('2')]
non_fighter_cols = [col for col in ml_df.columns if col not in red_cols + blue_cols + ['target_red']]

# 10.2. Crear el dataset 'FIGHTER_A' (Red Corner Original)
df_A = ml_df[non_fighter_cols + red_cols + blue_cols + ['target_red']].copy()
df_A = df_A.rename(columns={'target_red': 'target'})
# Renombrar columnas a un formato neutro (A y B)
mapping_A = {c: c.replace('_red', '_A').replace('_blue', '_B').replace('1', '_A').replace('2', '_B') for c in df_A.columns}
df_A = df_A.rename(columns=mapping_A)
df_A['is_red_corner'] = 1 # Feature para saber qué esquina era

# 10.3. Crear el dataset 'FIGHTER_B' (Blue Corner Invertido)
df_B = ml_df[non_fighter_cols + blue_cols + red_cols + ['target_red']].copy()
# Asegurar que no haya columnas duplicadas antes de operar
df_A = df_A.loc[:, ~df_A.columns.duplicated()]
df_B = df_B.loc[:, ~df_B.columns.duplicated()]

df_B['target'] = 1 - df_B['target_red'] # ¡INVERTIR TARGET! Si Red ganó (1), ahora Blue perdió (0)
df_B = df_B.drop(columns=['target_red'])

# Renombrar columnas (Intercambiando A y B)
mapping_B = {c: c.replace('_blue', '_A').replace('_red', '_B').replace('2', '_A').replace('1', '_B') for c in df_B.columns}
df_B = df_B.rename(columns=mapping_B)
df_B['is_red_corner'] = 0

# 10.4. Concatenar y Limpiar
ml_dataset = pd.concat([df_A, df_B], ignore_index=True)

# Eliminar columnas redundantes (IDs, Nombres)
cols_to_drop = ['red_id', 'blue_id', 'red_name', 'blue_name', 'winner_id', 'event_id']
ml_dataset = ml_dataset.drop(columns=[c for c in cols_to_drop if c in ml_dataset.columns], errors='ignore')

# Imputar nulos restantes (generalmente en stats de peleadores que no tenían registro)
for col in ml_dataset.columns:
    if ml_dataset[col].dtype in ['float64', 'int64']:
        # Usar 0 para stats de pelea (kd_A, sub_B, etc.)
        if any(stat in col for stat in stats_cols):
             ml_dataset[col] = ml_dataset[col].fillna(0)
        # Usar la mediana para stats de carrera (td_acc_A, slpm_B, etc.)
        else:
             ml_dataset[col] = ml_dataset[col].fillna(ml_dataset[col].median())
             
# Guardar el dataset final listo para ML
ml_dataset.to_csv('../data/ml/final_ufc_ml_dataset.csv', index=False)

print("\n✓ Dataset de ML final SIMÉTRICO generado y guardado en '../data/ml/final_ufc_ml_dataset.csv'")
print(f"Filas finales (simétricas): {len(ml_dataset)} | Columnas: {len(ml_dataset.columns)}")
print("¡El dataset está listo para el entrenamiento del modelo!")




✓ Dataset de ML final SIMÉTRICO generado y guardado en '../data/ml/final_ufc_ml_dataset.csv'
Filas finales (simétricas): 16406 | Columnas: 70
¡El dataset está listo para el entrenamiento del modelo!


In [48]:
# --- ML: Codificación de variables categóricas y bonus para el modelo ---
# 1. Codificar referee
ml_dataset['referee_code'] = ml_dataset['referee'].astype('category').cat.codes

# 2. Codificar weight_class, method, time_format
for col in ['weight_class', 'method', 'time_format']:
    ml_dataset[f'{col}_code'] = ml_dataset[col].astype('category').cat.codes

# 3. One-hot encoding de bonus (puede haber varios)
ml_dataset['bonus'] = ml_dataset['bonus'].fillna('').apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') else [])
for bonus_type in ['FIGHT', 'PERF', 'KO', 'SUB', 'BELT']:
    ml_dataset[f'bonus_{bonus_type.lower()}'] = ml_dataset['bonus'].apply(lambda x: int(bonus_type in x))

# 4. Eliminar columnas originales de texto y details
ml_dataset = ml_dataset.drop(columns=['referee', 'weight_class', 'method', 'time_format', 'bonus', 'details'], errors='ignore')

# Guardar el dataset final listo para ML (con variables numéricas)
ml_dataset.to_csv('../data/ml/final_ufc_ml_dataset.csv', index=False)

print("\n✓ Dataset de ML final codificado y guardado en '../data/ml/final_ufc_ml_dataset.csv'")


✓ Dataset de ML final codificado y guardado en '../data/ml/final_ufc_ml_dataset.csv'


In [49]:
# 10. Exportación del archivo limpio
# Guardar el DataFrame limpio
df.to_csv('../data/processed/fights.csv', index=False)